# Plots of stability regions

In [ ]:
import numpy as np
import scipy.linalg as linalg
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib as mpl

from nodepy import rk

from OrderCondition import *

import plot_fkt
plot_fkt.setup_plt()

mpl.rcParams['hatch.linewidth'] = 2.0

In [ ]:
def plot_stab(stab_function,color,fill=None,xmin=-10,xmax=10,yminmax=10,hatch=None):
    p,q = stab_function

    # Convert coefficients to floats for speed
    if p.coeffs.dtype=='object':
        p = np.poly1d([float(c) for c in p.coeffs])
    if q.coeffs.dtype=='object':
        q = np.poly1d([float(c) for c in q.coeffs])
        
    u = np.linspace(xmin,xmax,100)
    v = np.linspace(-yminmax,yminmax,100)
    
    U,V = np.meshgrid(u,v)
    Q = U+V*1j
    R=np.abs(p(Q)/q(Q))

    if fill:
        plt.pcolormesh(U, V, np.log(R),cmap=plt.get_cmap('seismic'),vmin=-4, vmax=4)
        plt.colorbar()
    if hatch:
        cs = plt.contourf(U,V,R,[0,1],hatches=[hatch],colors='none',alpha=1,extend='neither')
        cs.collections[0].set_ec(color)
        cs.collections[0].set_fc('none')

    plt.contour(U,V,R,[1],colors=[color],alpha=1,linewidths=3)
    plt.grid()
    
def calc_stab(rkm,bs):
    stab_functions = []
    for b in bs:
        display(b)
        rkm.b = b
        stab_functions.append(rkm.stability_function())
    return stab_functions

def calc_stab_exp(rkm,bs):
    stab_functions = []
    for b in bs:
        display(b)
        rkm.b = b
        stab_functions.append(rk.ExplicitRungeKuttaMethod(A=rkm.A,b=b).stability_function())
    return stab_functions

def color(chi,color1=[0,0,1],color2=[0,0,0.5]):
    return (chi*color1[0]+(1-chi)*color2[0],chi*color1[1]+(1-chi)*color2[1],chi*color1[2]+(1-chi)*color2[2])

# Implicit case

We are plotting a convex combination of original BE 3 extrapolation method and the embedded backward Euler scheme.

In [ ]:
ex3 = rk.extrap(3,'implicit euler').__num__()

chis = np.linspace(0,1,10)
bs = []
for chi in chis:
    bs.append(chi*ex3.b+(1-chi)*np.array([0,0,0,1/3,1/3,1/3]))


print('b[0]',bs[0])
print('b[-1]',bs[-1])    

stab_functions = calc_stab(ex3,bs)


In [ ]:
scale = 1.2
fig, ax = plt.subplots(1, 1, figsize=(scale*6.4, scale*4.8))

for i in range(1,9):
    plot_stab(stab_functions[i],(0.5,0.5,0.5))
    
plot_stab(stab_functions[0],'C0',hatch='/')
plot_stab(stab_functions[-1],'C1',hatch='\\')

be3 = mlines.Line2D([], [], color='C0', label='chain of BE')
ex3 = mlines.Line2D([], [], color='C1', label='BE 3 extrapolation')
altered = mlines.Line2D([], [], color=(0.5,0.5,0.5),label='convex combination')
legend_marker = plt.legend(handles=[be3,ex3,altered],loc='upper left')
plt.gca().add_artist(legend_marker)

plt.grid()
plt.savefig('stab_ex3.pdf', bbox_inches="tight")

# Explicit case

In [ ]:
ck5_or = rk.loadRKM('CK5').__num__()
ck5 = rk.ExplicitRungeKuttaMethod(A=ck5_or.A,b=ck5_or.b) #get rid of embedded method

(O,rhs)=OrderCond(ck5.A,ck5.c,order=4)
b_1 =linalg.null_space(O)[:,0]

chis = np.linspace(0,1,5)
alphas = np.linspace(-0.2,0.2,5)
bs = []
for alpha in alphas:
    bs.append(ck5.b+alpha*b_1)

stab_functions = calc_stab_exp(ck5,bs)

In [ ]:
for i in range(5):
    print(i)
    plot_stab(stab_functions[i],color(chis[i]),xmax=5,xmin=-5,yminmax=5)

plt.savefig('stab_ck5.pdf')

In [ ]:
b = np.load('b_ex_adde.npy') #Data from the notebook Convergence.ipynb

In [ ]:
b.shape

In [ ]:
bs = []
for i in range(0,b.shape[2],1):
    bs.append(b[:,0,i])

In [ ]:
bs

In [ ]:
dp5_or = rk.loadRKM('DP5').__num__()
dp5 = rk.ExplicitRungeKuttaMethod(A=dp5_or.A,b=dp5_or.b) #get rid of embedded method

stab_functions = calc_stab_exp(dp5,bs)

In [ ]:
scale = 1.2
fig, ax = plt.subplots(1, 1, figsize=(scale*6.4, scale*4.8))

for i in range(len(bs)):
    print(i)
    display(stab_functions[i])
    plot_stab(stab_functions[i],(0.5,0.5,0.5),xmax=6.5,xmin=-6.5,yminmax=5.5)

dp5 = rk.ExplicitRungeKuttaMethod(A=dp5_or.A,b=dp5_or.b)
stab_orig = dp5.stability_function()
plot_stab(stab_orig,'C0',xmax=5,xmin=-5,yminmax=5,hatch='/')
plt.grid(1)

original = mlines.Line2D([], [], color='C0', label='Dormand--Prince RK5')
altered = mlines.Line2D([], [], color=(0.5,0.5,0.5),label='free adaptation')
legend_marker = plt.legend(handles=[original,altered], loc="lower center", ncol=2, columnspacing=0.5)
plt.gca().add_artist(legend_marker)

plt.savefig('stab_dp5.pdf', bbox_inches="tight")

In [ ]:
dp5 = rk.ExplicitRungeKuttaMethod(A=dp5_or.A,b=dp5_or.b)
stab_orig = dp5.stability_function()
plot_stab(stab_orig,(0,0,0.5,1),xmax=5,xmin=-5,yminmax=5)

In [ ]:
dp5_1 = rk.ExplicitRungeKuttaMethod(A=dp5_or.A,b=bs[0])
stab_1 = dp5_1.stability_function()
plot_stab(stab_1,(0,0,0.5,1),xmax=5,xmin=-5,yminmax=5)

In [ ]:
plot_stab(stab_orig,(0,0,0.5,1),xmax=4,xmin=-4,yminmax=5)
plot_stab(stab_1,(0,0,0.5,1),xmax=4,xmin=-4,yminmax=5)

In [ ]:
plot_stab(stab_functions[0],(0.5,0.5,0.5),xmax=5,xmin=-5,yminmax=5)

In [ ]:
#It needed a bit of trickery to change the hatch color. I am leaving the idea here
u = np.linspace(-2,2,500)
v = np.linspace(-2,2,500)
    
U,V = np.meshgrid(u,v)
Q = U+V*1j
R=np.abs(Q)

In [ ]:
cs = plt.contourf(U,V,R,[0,1],hatches=['-'],colors=[(0,0,0,0)],alpha=1,extend='neither')
cs.collections[0].set_ec('blue')#Hatch color is tied to edge color
cs.collections[0].set_fc('none')#Make the rest transparent
cs.collections[0].set_lw(10)